# Data Preprocessing

## Missing Data

In [24]:
import numpy as np
import pandas as pd
from io import StringIO
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

### Create a mock CSV with missing data

In [2]:
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
print(df)

      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN


### Analyzing missing data

In [3]:
# Validates how many rows have missing values per column
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

## Handling missing data by removing

In [4]:
print(df.dropna(axis=0)) # Drops rows with missing values
print()

print(df.dropna(axis=1)) # Drops columns with missing values
print()

print(df.dropna(how='all')) # Drops rows where all columns are NaN
print()

print(df.dropna(thresh=4)) # Drops rows that have less than 4 real values
print()

print(df.dropna(subset=['C'])) # Drops rows where NaN appears in specific columns

     A    B    C    D
0  1.0  2.0  3.0  4.0

      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0

      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN

     A    B    C    D
0  1.0  2.0  3.0  4.0

      A     B     C    D
0   1.0   2.0   3.0  4.0
2  10.0  11.0  12.0  NaN


Dropping missing values is definitely the easiest way to handle data cleanup, but it does come with its downsides. As we remove columns, there is potential to remove features that contain valuable information. Also if we remove enough rows we can severly degrade our training data.

## Imputing missing data

### Mean Imputation

In [5]:
# Replace missing values with the mean of the column
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values) # Although dataframes are supported by sklearn, the API has matured more to work with numpy arrays

imputed_data = imr.transform(df.values)
print(imputed_data)

[[ 1.   2.   3.   4. ]
 [ 5.   6.   7.5  8. ]
 [10.  11.  12.   6. ]]


### Other forms of imputation

In [6]:
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
print(imputed_data)
print()

imr = SimpleImputer(missing_values=np.nan, strategy='median')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
print(imputed_data)

[[ 1.  2.  3.  4.]
 [ 5.  6.  3.  8.]
 [10. 11. 12.  4.]]

[[ 1.   2.   3.   4. ]
 [ 5.   6.   7.5  8. ]
 [10.  11.  12.   6. ]]


### Imputation Using Pandas

In [7]:
df.fillna(df.mean()) # Fills missing values with the mean of the column

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


The sklearn estimator API used in the SimpleImputer class is a type of transformer that is fitted to the training data. It can then be used to transform the training data and any future data (of the same size) that we want to predict on.

Why use the same transformer on different datasets?

- The reason we want to use the same transformer on different datasets is for consistency across data that will be used for training a model.
- Fitting a transformer once can be more efficient for larger datasets.
- Transfer of knowledge, the information learned from one dataset can be applied to another dataset.

## Categorical Data

In [8]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']
])

# Nominal feature [color]
# Ordinal feature [size]
# Numerical feature [price]
df.columns = ['color', 'size', 'price', 'classlabel']

df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### Mapping Ordinal Data

In [9]:
size_mapping = {
  'XL': 3,
  'L': 2,
  'M': 1
}

df['size'] = df['size'].map(size_mapping)

df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [10]:
# Later on we can reverse map it using an inverse map

inv_size_mapping = {v: k for k, v in size_mapping.items()}

df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### Encoding Class Labels

In [11]:
# Since class labels are not ordinal, the value we assign does not matter
class_mapping = { label: idx for idx, label in enumerate(np.unique(df['classlabel'])) }

class_mapping

{'class1': 0, 'class2': 1}

In [12]:
df['classlabel'] = df['classlabel'].map(class_mapping)

df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [13]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}

df['classlabel'] = df['classlabel'].map(inv_class_mapping)

df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [14]:
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [15]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### One-Hot Encoding Nominal Data

At first glance, it might make sense to encode nominal features similar to how we encoded the class labels above. While this may work, it is not optimical. One of the downsides to this is that encoding nominal data in an ascending numerical order can lead the model to believe a feature is "larger" than the other, that does not make sense in this case (for example blue being larger than red).

In [16]:
X = df[['color', 'size', 'price']].values

color_le = LabelEncoder()

X[:, 0] = color_le.fit_transform(X[:, 0])

X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In these instances, we want to use a method called one-hot encoding. This method creates new "dummy" columns that can represent the different categories in the nominal feature without any ordinal relationship.

In [17]:
X = df[['color', 'size', 'price']].values

color_ohe = OneHotEncoder()

color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [18]:
X = df[['color', 'size', 'price']].values

# Modify only the first column & leave the other two columns untouched using the 'passthrough' argument
c_transf = ColumnTransformer([
    ('onehot', OneHotEncoder(), [0]),
    ('nothing', 'passthrough', [1, 2])
])

c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [19]:
# Another method is using pandas get_dummies
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,False,True,False
1,13.5,2,False,False,True
2,15.3,3,True,False,False


One thing to look out for when using one-hot encoding is multicollinearity. This usually happens when multiple features are highly correlated. The reason this can be an issue, is that some models (like linear regression) do not understand that these highly correlated features are actually the same thing and will try to fit the model using both features. This can lead to overfitting. A couple ways to combat multicollinearity is to remove one of the features or use some type of dimensionality reduction technique.

In [20]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True) # Drops the first column to avoid multicollinearity

,price,size,color_green,color_red
0,10.1,1,True,False
1,13.5,2,False,True
2,15.3,3,False,False


In [21]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
    ('onehot', color_ohe, [0]),
    ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X).astype(float)

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

### Encoding Ordinal Data (Optional: Threshold Encoding)

In [22]:
df_2 = pd.DataFrame([
    ['green', 'M', 10.1],
    ['red', 'L', 13.5],
    ['blue', 'XL', 15.3]
])

df_2.columns = ['color', 'size', 'price']

df_2['x > M'] = df_2['size'].apply(lambda x: 1 if x in ['L', 'XL'] else 0)
df_2['x > L'] = df_2['size'].apply(lambda x: 1 if x == 'XL' else 0)
del df_2['size']

df_2

,color,price,x > M,x > L
0,green,10.1,0,0
1,red,13.5,1,0
2,blue,15.3,1,1


Encoding ordinal data using threshold encoding is helpful if we do not know the exact differences between two ordinal values.